In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import catboost as cb
from catboost import CatBoostClassifier
import time

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score, average_precision_score, precision_score, recall_score, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold

import optuna
from optuna.integration import CatBoostPruningCallback

from cc4_preprocessor import Preprocessor

np.random.seed(42)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('data/Training_TriGuard.csv')
df = df.dropna(subset=['subrogation'])

In [3]:
pre = Preprocessor(smoothing_factor=5, mode = 'catboost')

In [4]:
X = df.drop(columns=["subrogation"]).copy()
y = df["subrogation"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=0)

In [5]:
y_train.value_counts(normalize=True)

subrogation
0.0    0.77141
1.0    0.22859
Name: proportion, dtype: float64

In [6]:
y_test.value_counts(normalize=True)

subrogation
0.0    0.771296
1.0    0.228704
Name: proportion, dtype: float64

In [7]:
pre.fit(X_train, y_train)

X_train_proc = pre.transform(X_train)
X_test_proc = pre.transform(X_test)

X_test_proc = X_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)

Fitting Preprocessor in 'catboost' mode...
CatBoost mode: Skipping target encoding learning.
Fit complete.
Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.


In [8]:
CAT_FEATURES = [X_train_proc.columns.get_loc(c)
                for c in pre.cat_for_encoding_ if c in X_train_proc.columns]
print("Cat features used:", [X_train_proc.columns[i] for i in CAT_FEATURES])

Cat features used: ['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season', 'zip3']


## CatBoost with Optuna Tuning

In [9]:
def objective(trial):
    params = {
        "iterations": 4000,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 20.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "random_strength": trial.suggest_float("random_strength", 1e-8, 1.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "border_count": trial.suggest_int("border_count", 64, 255),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 0.5, 10.0, log=True),
        "loss_function": "Logloss",
        "eval_metric": "F1",       # monitor F1, but WE compute F1 with predict (0.5)
        "random_seed": 42,
        "task_type": "CPU",
        "early_stopping_rounds": 200,
        "verbose": False,
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    f1s = []
    for train_idx, valid_idx in skf.split(X_train_proc, y_train):
        Xtr, Xva = X_train_proc.iloc[train_idx], X_train_proc.iloc[valid_idx]
        ytr, yva = y_train.iloc[train_idx], y_train.iloc[valid_idx]

        model = cb.CatBoostClassifier(**params)
        model.fit(
            Xtr, ytr,
            eval_set=(Xva, yva),
            cat_features=CAT_FEATURES,
            use_best_model=True,
            verbose=False
        )

        # IMPORTANT: use class predictions (threshold=0.5) to mirror the competition
        yva_hat = model.predict(Xva)  # returns {0,1}
        f1s.append(f1_score(yva, yva_hat))

    return float(np.mean(f1s))

In [10]:
study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=8)
)
study.optimize(objective, n_trials=60, show_progress_bar=True)

print("Best CV F1:", study.best_value)
print("Best params:", study.best_trial.params)

# Fit final on all training data with best params
best_params = study.best_trial.params.copy()
final_params = {
    "iterations": 4000,
    "loss_function": "Logloss",
    "eval_metric": "F1",
    "random_seed": 42,
    "task_type": "CPU",
    "early_stopping_rounds": 300,
    "verbose": False,
    **best_params
}

final_model = cb.CatBoostClassifier(**final_params)
final_model.fit(
    X_train_proc, y_train,
    eval_set=(X_test_proc, y_test),  # for monitoring only
    cat_features=CAT_FEATURES,
    use_best_model=True,
    verbose=False
)

# Final test evaluation (THRESHOLD=0.5 by default)
ytest_hat = final_model.predict(X_test_proc)
print("Final TEST F1:", f1_score(y_test, ytest_hat))
print(classification_report(y_test, ytest_hat, digits=4))

[I 2025-11-06 22:13:36,156] A new study created in memory with name: no-name-97b6305e-fab6-4872-a211-d2e201d3041e
Best trial: 0. Best value: 0.556399:   2%|▏         | 1/60 [00:13<13:15, 13.49s/it]

[I 2025-11-06 22:13:49,651] Trial 0 finished with value: 0.5563986004523326 and parameters: {'learning_rate': 0.02806504517490617, 'depth': 7, 'l2_leaf_reg': 0.01083612654191987, 'subsample': 0.6663386434069978, 'random_strength': 4.560475471280003e-07, 'bagging_temperature': 0.8073750318210177, 'border_count': 102, 'scale_pos_weight': 5.196436487035085}. Best is trial 0 with value: 0.5563986004523326.


Best trial: 0. Best value: 0.556399:   3%|▎         | 2/60 [00:29<14:35, 15.10s/it]

[I 2025-11-06 22:14:05,877] Trial 1 finished with value: 0.5418739476157822 and parameters: {'learning_rate': 0.012107568399045572, 'depth': 7, 'l2_leaf_reg': 15.971708506369344, 'subsample': 0.6363163907807493, 'random_strength': 0.00022163764173885748, 'bagging_temperature': 0.4420300352514631, 'border_count': 201, 'scale_pos_weight': 6.800651553904118}. Best is trial 0 with value: 0.5563986004523326.


Best trial: 0. Best value: 0.556399:   5%|▌         | 3/60 [00:41<12:50, 13.52s/it]

...

[I 2025-11-06 22:26:14,447] Trial 57 finished with value: 0.5957389349404557 and parameters: {'learning_rate': 0.05651084698349054, 'depth': 4, 'l2_leaf_reg': 0.2005890577083515, 'subsample': 0.7486121688985079, 'random_strength': 2.876768589712275e-06, 'bagging_temperature': 0.7716805251999412, 'border_count': 149, 'scale_pos_weight': 2.629471478283962}. Best is trial 52 with value: 0.5980844730134971.


Best trial: 52. Best value: 0.598084:  98%|█████████▊| 59/60 [12:46<00:09,  9.06s/it]

[I 2025-11-06 22:26:22,752] Trial 58 finished with value: 0.5728013463740338 and parameters: {'learning_rate': 0.05342659131638298, 'depth': 4, 'l2_leaf_reg': 0.6130975854052565, 'subsample': 0.8819049812223653, 'random_strength': 2.4502956187033664e-07, 'bagging_temperature': 0.23769469408103827, 'border_count': 140, 'scale_pos_weight': 1.4949305925108214}. Best is trial 52 with value: 0.5980844730134971.


Best trial: 52. Best value: 0.598084: 100%|██████████| 60/60 [12:57<00:00, 12.96s/it]


[I 2025-11-06 22:26:33,900] Trial 59 finished with value: 0.5836707154822135 and parameters: {'learning_rate': 0.04199923592974078, 'depth': 6, 'l2_leaf_reg': 0.0400914663496354, 'subsample': 0.815160388710766, 'random_strength': 1.740458827376656e-06, 'bagging_temperature': 0.6018855922016728, 'border_count': 163, 'scale_pos_weight': 3.168659259981703}. Best is trial 52 with value: 0.5980844730134971.
Best CV F1: 0.5980844730134971
Best params: {'learning_rate': 0.11361923965435601, 'depth': 4, 'l2_leaf_reg': 0.7332789450189878, 'subsample': 0.783908770635686, 'random_strength': 6.564014768505255e-08, 'bagging_temperature': 0.45750300497470253, 'border_count': 132, 'scale_pos_weight': 2.600346356658591}
Final TEST F1: 0.5970149253731343
              precision    recall  f1-score   support

         0.0     0.9113    0.7774    0.8391      4165
         1.0     0.4981    0.7449    0.5970      1235

    accuracy                         0.7700      5400
   macro avg     0.7047    0.7612 

In [ ]:
# Output module, from model_citizens.ipynb
real_test = pd.read_csv("data/Testing_TriGuard.csv")

X_real_test_proc = pre.transform(real_test)
X_real_test_proc = X_real_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)
real_pred_proba = best_model.predict_proba(X_real_test_proc)[:, 1]
real_pred_label = (real_pred_proba >= 0.5).astype(int)

prediction = pd.DataFrame({
    "claim_number": real_test["claim_number"],
    "subrogation": real_pred_label
})

print(prediction.head())

Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
   claim_number  subrogation
0       3126034            0
1       7380142            1
2       4655051            0
3       6728725            1
4       9848460            1


In [ ]:
# prediction.to_csv("results/catboost_2c_prediction.csv", index=False)

In [29]:
importances = best_model.get_feature_importance()
feature_names = best_model.feature_names_

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
})

feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print("--- Feature Importance Analysis ---")
print(f"Total features: {len(feature_importance_df)}")

print("\nTop 10 Most Important Features:")
print(feature_importance_df.head(10))

print("\nBottom 10 Least Important Features:")
print(feature_importance_df.tail(10))

n_features_to_remove = 100
n_features_to_keep = len(feature_importance_df) - n_features_to_remove
top_features = feature_importance_df.head(n_features_to_keep)['feature'].tolist()

print(f"\nKeeping top {len(top_features)} features and removing bottom {n_features_to_remove}.")

X_train_top_features = X_train_proc[top_features]
X_test_top_features = X_test_proc.reindex(columns=top_features, fill_value=0) 

original_cat_features = set(CAT_FEATURES)
top_features_set = set(top_features)
new_cat_features = list(original_cat_features.intersection(top_features_set))

print(f"Original categorical features: {len(CAT_FEATURES)}")
print(f"Categorical features kept: {len(new_cat_features)}")

best_params_from_optuna = study.best_trial.params

final_params_new = best_params_from_optuna.copy()
final_params_new.update({
    'iterations': 2000, 
    'eval_metric': 'Logloss',
    'task_type': 'CPU',
    'early_stopping_rounds': 50
})

print("\nRetraining model with top features...")
new_model = CatBoostClassifier(**final_params_new)

new_model.fit(
    X_train_top_features, y_train,
    eval_set=(X_test_top_features, y_test),
    cat_features=new_cat_features,
    verbose=False
)

y_preds_new = new_model.predict(X_test_top_features)
new_f1 = f1_score(y_test, y_preds_new, pos_label=1)

print("\n--- Model Performance Comparison ---")
# 'final_f1' comes from cell 30 in your notebook
print(f"Original F1 score (all features): {final_f1:.4f}")
print(f"New F1 score (top {len(top_features)} features): {new_f1:.4f}")

print("\nNew Model Classification Report (Top Features):")
print(classification_report(y_test, y_preds_new, target_names=['Class 0.0', 'Class 1.0']))

--- Feature Importance Analysis ---
Total features: 136

Top 10 Most Important Features:
                        feature  importance
44              liab_x_multicar   11.261667
135  recovery_feasibility_score    9.755106
58     is_multi_vehicle_unclear    4.774472
53               liab_prct_sqrt    4.758176
51            liab_prct_squared    4.595245
55                 liab_inverse    4.462763
13                    liab_prct    4.366832
45         liab_x_highrisk_site    4.164962
54                liab_prct_log    3.882838
50          multicar_x_highrisk    3.744198

Bottom 10 Least Important Features:
                   feature  importance
87          luxury_vehicle         0.0
89         economy_vehicle         0.0
91           light_vehicle         0.0
94            high_mileage         0.0
59           is_single_car         0.0
100  very_frequent_claimer         0.0
102          medium_payout         0.0
103           small_payout         0.0
104      very_large_payout         0.0
